In [1]:
import pandas as pd
import numpy as np
import sklearn 
import matplotlib.pyplot as plt
import os
import factoroid as fr
import sys
import torch
import shutil
import json
import datetime as dt
from datetime import timedelta
from scipy import interpolate
WebSQL = fr.connector.MySQLDB('db-01')
DevSQL = fr.connector.MySQLDB('db-dev')
import pickle

Successfully connected; Factorid web services
Successfully connected; Factoroid development


In [2]:
query_cycle = '''
SELECT
    *
FROM 
    yk_operating_cycle
WHERE
    equipment_name in ('ECOSPEED F1040 #2')

'''
pdf_cycle = pd.DataFrame(WebSQL.query(query_cycle))

In [4]:
with open('./parameter/equipment_properties.json', 'r') as param_file:
    parameters = json.load(param_file) 
machine_mac_list = []
for machine in parameters.keys():
    if machine != 'door_sensor_list':
        machine_mac_list.append([machine, parameters[machine]['mac']])
pdf_mac = pd.DataFrame(machine_mac_list, columns=["machine", "mac"])

In [5]:
pdf_result = pd.merge(pdf_cycle, pdf_mac, how='inner', left_on='equipment_name', right_on='machine')

In [8]:
def normalizer(df, length):
    df['timestamp'] = df['timestamp'] - df['timestamp'].min()
    df['timestamp'] = df.apply(lambda x : x['timestamp'].total_seconds(), axis=1)
    df['timestamp'] = df['timestamp']/df['timestamp'].max()
    df['value'] = (df['value'] - df['value'].mean()) / df['value'].std()
    f = interpolate.interp1d(df['timestamp'], df['value'], kind='linear')
    x = np.linspace(0, 1, 1001)
    y = f(x)
    return y.T

In [9]:
length = 1001
samples = {}
dataset = pd.DataFrame(columns = [i for i in range(length)])
for i, row in pdf_result[['id', 'mac', 'start_time', 'stop_time', 'gop']].iterrows():
    if row.mac not in samples:
        samples[row.mac] = []
    period_ = fr.utils.Period([row.start_time.strftime('%Y-%m-%d %H:%M:%S'), row.stop_time.strftime('%Y-%m-%d %H:%M:%S')])
    data = fr.processor.preprocess(WebSQL.get_data(mac=row.mac, period=period_.utc))[['timestamp', 'value']]
    y = normalizer(data, length)
    row_out = [row.id] + list(y) + [row.gop]
    samples[row.mac].append(row_out)

In [10]:
test_data = pd.DataFrame(samples['D72B9E3889F1'], columns=['id'] + ['x%d'%(i) for i in range(length)] + ['label'])
X = test_data.drop(columns=['id','label'])
y = test_data['label']

In [13]:
# test_data2 = pd.concat([test_data.drop(columns=['label']), pd.get_dummies(test_data['label'])], axis=1)
# X = test_data2.drop(columns=['id', '10', '20', '30'])
# y = test_data2[['10', '20', '30']]

In [14]:
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

In [15]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25)

In [16]:
clf = MLPClassifier(hidden_layer_sizes=(150,100,50), max_iter=300, activation = 'relu',solver='adam')
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)

In [17]:
accuracy_score(y_test, y_pred)

0.9473684210526315